In [1]:
import pymysql
import pandas as pd
from surprise import KNNBasic, SVD, SVDpp, NMF, Reader, Dataset
from surprise.model_selection import cross_validate

In [19]:
con = pymysql.connect(host = '127.0.0.1', user = 'root', password = 'kyjin0413!',
                       db = 'lpointsample', charset = 'utf8')
cur = con.cursor(pymysql.cursors.DictCursor)
cur.execute(
    "select de_dt as 날짜, de_hr as 시간대, sum(buy_am) as 금액 ,count(*) as 이용횟수 from 엘페이이용_view group by de_dt,de_hr order by de_dt,de_hr")
result=cur.fetchall()
con.commit()
con.close()
df= pd.DataFrame(result)
df.columns=['일자','시간대', '금액','이용횟수']
df = df.set_index(['일자'])
df

# 날짜랑 금액 select de_dt as 날짜 , sum(buy_am) as 금액 from 엘페이이용_view group by de_dt order by de_dt
# 시간대랑 금액 select de_hr as 시간대, sum(buy_am) as 금액 from 엘페이이용_view group by de_day, de_hr order by de_day

,시간대,금액,이용횟수
일자,,,
2021-01-01,0,2683380,72
2021-01-01,1,499360,11
2021-01-01,2,64176,10
2021-01-01,3,885833,11
2021-01-01,5,37630,1
...,...,...,...
2021-12-31,19,5136390,63
2021-12-31,20,3124749,72
2021-12-31,21,1210011,33


In [3]:
con = pymysql.connect(host = '127.0.0.1', user = 'root', password = 'kyjin0413!',
                       db = 'lpointsample', charset = 'utf8')
cur = con.cursor(pymysql.cursors.DictCursor)
cur.execute("select de_dt as 날짜, de_hr as 시간대, sum(buy_am) as 금액 ,count(*) as 이용횟수 from 엘페이이용_view group by de_dt,de_hr order by de_dt,de_hr;")
result = cur.fetchall()
con.commit()
con.close()
df = pd.DataFrame(result)
df.columns=['일자','시간대', '금액','이용횟수']
for i in range(len(df)):
    a = str(df['일자'][i])[:10]
    b = str(df['시간대'][i])
    if len(b) == 1:
        b= "0"+b
    df['일자'][i]=a+" "+b
    df['일자'][i] = pd.to_datetime(df['일자'][i], format='%Y-%m-%d %H')

df=df.drop(columns=['시간대'])

C:\Users\ASIA-08\AppData\Local\Temp\ipykernel_12576\1892269504.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['일자'][i]=a+" "+b
C:\Users\ASIA-08\AppData\Local\Temp\ipykernel_12576\1892269504.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['일자'][i] = pd.to_datetime(df['일자'][i], format='%Y-%m-%d %H')


In [11]:
df

,소비 금액,이용 횟수
일자,,
2021-01-01 00:00:00,2683380,72
2021-01-01 01:00:00,499360,11
2021-01-01 02:00:00,64176,10
2021-01-01 03:00:00,885833,11
2021-01-01 05:00:00,37630,1
...,...,...
2021-12-31 19:00:00,5136390,63
2021-12-31 20:00:00,3124749,72
2021-12-31 21:00:00,1210011,33


In [33]:
data = pd.read_csv('전체1.csv', encoding = 'cp949')
data = data.set_index(['일자'])
data = data.drop(['소비 금액', '이용 횟수'], axis = 1)
data = data.fillna(0)
data

,평균온도,평균습도
일자,,
2021-01-01 0:00,-8.450000,51.642857
2021-01-01 1:00,-8.907143,52.000000
2021-01-01 2:00,-9.528571,55.071429
2021-01-01 3:00,-10.178571,56.500000
2021-01-01 4:00,-10.578571,57.571429
...,...,...
2022-12-31 19:00,0.000000,0.000000
2022-12-31 20:00,0.000000,0.000000
2022-12-31 21:00,0.000000,0.000000


In [32]:
data = pd.read_csv('시간대전체(지역별).csv', encoding = 'cp949')
data = data.set_index(['일자'])
data = data.drop(['소비금액'], axis = 1)
data = data.fillna(0)
data

,강원_온도,강원_습도,경기_온도,경기_습도,경남_온도,경남_습도,전북_온도,전북_습도,제주_온도,제주_습도,인천_온도,인천_습도
일자,,,,,,,,,,,,
2021-01-01 0:00,-6.3,30.0,-10.0,63.0,-2.2,59.0,4.5,53.0,-6.0,91.0,-7.9,73.0
2021-01-01 1:00,-6.7,30.0,-11.2,71.0,-3.1,68.0,4.3,58.0,-5.7,91.0,-9.2,76.0
2021-01-01 2:00,-6.7,26.0,-11.6,76.0,-4.1,61.0,4.2,54.0,-6.0,92.0,-10.6,81.0
2021-01-01 3:00,-7.2,25.0,-12.0,78.0,-4.7,58.0,4.3,52.0,-6.5,91.0,-10.7,83.0
2021-01-01 4:00,-7.6,25.0,-12.5,81.0,-5.2,58.0,4.2,47.0,-8.3,90.0,-11.0,82.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 19:00,-7.2,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-12-31 20:00,-7.8,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-12-31 21:00,-8.4,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


--------------------------------------------------------------------------------------